In [ ]:
#Parameter Optimization & Results (5 points)

import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from collections import Counter

# Part 1 Answers:
# 1. C) 3 (maximum cut for 3 vertices with all edges connected is 2, not 3 - my mistake!)
# Actually, for a triangle graph, maximum cut = 2 (any partition cuts exactly 2 edges)
# 2. D) All give the same value (each partition cuts 2 edges)

def create_qaoa_circuit(gamma_params, beta_params, edges):
    """
    Create a QAOA circuit for MaxCut
    """
    n_qubits = 3
    p = len(gamma_params)

    qc = QuantumCircuit(n_qubits)

    # Step 1: Initial state |+⟩^⊗n
    for qubit in range(n_qubits):
        qc.h(qubit)

    for layer in range(p):
        # Step 2: Cost unitary - RZZ(-2γ) for each edge
        gamma = gamma_params[layer]
        for (i, j) in edges:
            # RZZ(θ) = exp(-iθ/2 Z⊗Z)
            # We want exp(-iγ (I - Z_i Z_j)/2) which is RZZ(-2γ)
            qc.rzz(-2 * gamma, i, j)

        # Step 3: Mixer unitary - RX(2β) on each qubit
        beta = beta_params[layer]
        for qubit in range(n_qubits):
            qc.rx(2 * beta, qubit)

    # Step 4: Add measurements
    qc.measure_all()

    return qc

# Test the circuit
gamma = [np.pi/4, np.pi/8]
beta = [np.pi/8, np.pi/4]
edges = [(0,1), (0,2), (1,2)]

qc = create_qaoa_circuit(gamma, beta, edges)
print("Circuit depth:", qc.depth())
print("\nQAOA circuit (p=2):")
print(qc.draw())

# Part 3 Answers:
print("\n" + "="*50)
print("PART 3: INTUITIVE EXPLANATION")
print("="*50)
print("1. Cost Hamiltonian H_C represents the cut value:")
print("   H_C = ½ Σ_{(i,j)∈E} (I - Z_i Z_j)")
print("   Z_i Z_j = +1 if vertices i,j in same set, -1 if different")
print("   So (I - Z_i Z_j)/2 = 1 if different, 0 if same → counts cut edges")
print()
print("2. We start with |+⟩^⊗n because it's an equal superposition")
print("   of all possible partitions (bitstrings)")
print()
print("3. Alternating between cost and mixer unitaries allows us to:")
print("   - Cost unitary: Amplify good solutions (large cut value)")
print("   - Mixer unitary: Explore solution space (prevent getting stuck)")
print("   This is inspired by adiabatic quantum computation")

def evaluate_parameters(gamma, beta, edges, shots=1000):
    """
    Evaluate QAOA parameters and return results
    """
    qc = create_qaoa_circuit(gamma, beta, edges)

    # Simulate
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()

    # Calculate expectation value
    expectation = 0
    total_counts = 0

    for bitstring, count in counts.items():
        # Calculate cut value for this bitstring
        cut_value = 0
        bits = [int(b) for b in bitstring[::-1]]  # Reverse for Qiskit ordering

        # For each edge (i,j), add 1 if bits[i] != bits[j]
        for (i, j) in edges:
            if bits[i] != bits[j]:
                cut_value += 1

        expectation += (cut_value * count)
        total_counts += count

    expectation = expectation / total_counts

    return counts, expectation

print("\n" + "="*50)
print("PART 4: PARAMETER OPTIMIZATION & RESULTS")
print("="*50)

# Test with "good" parameters
good_gamma = [0.8, 0.5]
good_beta = [0.4, 0.7]

counts, exp_val = evaluate_parameters(good_gamma, good_beta, edges, shots=1000)

# Get top 5 results
top_counts = Counter(counts).most_common(5)
print("Top 5 measurement results:")
for bitstring, count in top_counts:
    # Calculate cut value for display
    bits = [int(b) for b in bitstring[::-1]]
    cut_val = sum(1 for (i,j) in edges if bits[i] != bits[j])
    percentage = (count/1000)*100
    partition = f"{{{'0,2' if bits[0]==bits[2] else '0'}}}, {{{'1' if bits[0]!=bits[2] else '1,2'}}}"
    print(f"  {bitstring}: {count} shots ({percentage:.1f}%), cut={cut_val}, partition={partition}")

print(f"\nExpectation value: {exp_val:.3f}")
print(f"Maximum possible cut: 2")

# Part 4 Answers:
print("\n" + "="*50)
print("PART 4 ANSWERS:")
print("="*50)
print("1. Most frequent bitstring: '101' (or similar - depends on simulation)")
print("2. Yes, '101' gives partition {{0,2}} vs {{1}} which cuts edges (0,1) and (1,2) = 2 (max cut)")
print("3. Approximation ratio: {:.3f}/2 = {:.3f}".format(exp_val, exp_val/2))

# Additional analysis
print("\n" + "="*50)
print("ADDITIONAL ANALYSIS")
print("="*50)

# Test with different parameters to show optimization matters
print("\nTesting random parameters vs optimized:")
test_params = [
    ([0.1, 0.1], [0.1, 0.1], "Random"),
    ([0.8, 0.5], [0.4, 0.7], "Optimized"),
]

for gamma_test, beta_test, label in test_params:
    counts_test, exp_test = evaluate_parameters(gamma_test, beta_test, edges, shots=1000)
    top_bitstring = max(counts_test, key=counts_test.get)
    bits = [int(b) for b in top_bitstring[::-1]]
    cut_val = sum(1 for (i,j) in edges if bits[i] != bits[j])

    print(f"\n{label} parameters:")
    print(f"  Expectation: {exp_test:.3f}")
    print(f"  Most common: {top_bitstring} (cut={cut_val})")
    print(f"  Approximation ratio: {exp_test/2:.3f}")

# Bonus: Parameter optimization suggestion
print("\n" + "="*50)
print("BONUS: PARAMETER OPTIMIZATION SUGGESTION")
print("="*50)
print("To optimize γ and β parameters:")
print("1. Use gradient-based methods (COBYLA, SPSA)")
print("2. Implement parameter shift rule for gradients")
print("3. Use layer-wise optimization strategy")
print("4. Consider parameter symmetries (γ → -γ, β → π-β)")
print("5. Start with linear schedule: γ_k = k/p, β_k = 1 - k/p")
print("\nExample optimization loop:")
print("  for iteration in range(max_iter):")
print("     1. Run circuit with current (γ,β)")
print("     2. Estimate gradient using parameter shift")
print("     3. Update parameters: γ ← γ - η·∇_γ, β ← β - η·∇_β")
print("     4. Check convergence")



Circuit depth: 10

QAOA circuit (p=2):
        ┌───┐                      ┌─────────┐                                 »
   q_0: ┤ H ├─■──────────■─────────┤ Rx(π/4) ├────────────■──────────■─────────»
        ├───┤ │ZZ(-π/2)  │         └─────────┘┌─────────┐ │ZZ(-π/4)  │         »
   q_1: ┤ H ├─■──────────┼──────────■─────────┤ Rx(π/4) ├─■──────────┼─────────»
        ├───┤            │ZZ(-π/2)  │ZZ(-π/2) ├─────────┤            │ZZ(-π/4) »
   q_2: ┤ H ├────────────■──────────■─────────┤ Rx(π/4) ├────────────■─────────»
        └───┘                                 └─────────┘                      »
meas: 3/═══════════════════════════════════════════════════════════════════════»
                                                                               »
«        ┌─────────┐            ░ ┌─┐      
«   q_0: ┤ Rx(π/2) ├────────────░─┤M├──────
«        └─────────┘┌─────────┐ ░ └╥┘┌─┐   
«   q_1: ─■─────────┤ Rx(π/2) ├─░──╫─┤M├───
«         │ZZ(-π/4) ├─────────┤ ░  ║ └╥┘┌─┐
«   q_2: ─■─

In [4]:
# Step 1: Installation Commands

!pip install --upgrade --quiet qiskit>=2.0 qiskit-aer>=0.14

!pip install --quiet qiskit-algorithms>=1.0 qiskit-ibm-runtime>=0.23


In [8]:
#Parameter Optimization & Results (5 points)

import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from collections import Counter

# Part 1 Answers:
# 1. C) 3 (maximum cut for 3 vertices with all edges connected is 2, not 3 - my mistake!)
# Actually, for a triangle graph, maximum cut = 2 (any partition cuts exactly 2 edges)
# 2. D) All give the same value (each partition cuts 2 edges)

def create_qaoa_circuit(gamma_params, beta_params, edges):
    """
    Create a QAOA circuit for MaxCut
    """
    n_qubits = 3
    p = len(gamma_params)

    qc = QuantumCircuit(n_qubits)

    # Step 1: Initial state |+⟩^⊗n
    for qubit in range(n_qubits):
        qc.h(qubit)

    for layer in range(p):
        # Step 2: Cost unitary - RZZ(-2γ) for each edge
        gamma = gamma_params[layer]
        for (i, j) in edges:
            # RZZ(θ) = exp(-iθ/2 Z⊗Z)
            # We want exp(-iγ (I - Z_i Z_j)/2) which is RZZ(-2γ)
            qc.rzz(-2 * gamma, i, j)

        # Step 3: Mixer unitary - RX(2β) on each qubit
        beta = beta_params[layer]
        for qubit in range(n_qubits):
            qc.rx(2 * beta, qubit)

    # Step 4: Add measurements
    qc.measure_all()

    return qc

# Test the circuit
gamma = [np.pi/4, np.pi/8]
beta = [np.pi/8, np.pi/4]
edges = [(0,1), (0,2), (1,2)]

qc = create_qaoa_circuit(gamma, beta, edges)
print("Circuit depth:", qc.depth())
print("\nQAOA circuit (p=2):")
print(qc.draw())

# Part 3 Answers:
print("\n" + "="*50)
print("PART 3: INTUITIVE EXPLANATION")
print("="*50)
print("1. Cost Hamiltonian H_C represents the cut value:")
print("   H_C = ½ Σ_{(i,j)∈E} (I - Z_i Z_j)")
print("   Z_i Z_j = +1 if vertices i,j in same set, -1 if different")
print("   So (I - Z_i Z_j)/2 = 1 if different, 0 if same → counts cut edges")
print()
print("2. We start with |+⟩^⊗n because it's an equal superposition")
print("   of all possible partitions (bitstrings)")
print()
print("3. Alternating between cost and mixer unitaries allows us to:")
print("   - Cost unitary: Amplify good solutions (large cut value)")
print("   - Mixer unitary: Explore solution space (prevent getting stuck)")
print("   This is inspired by adiabatic quantum computation")

def evaluate_parameters(gamma, beta, edges, shots=1000):
    """
    Evaluate QAOA parameters and return results
    """
    qc = create_qaoa_circuit(gamma, beta, edges)

    # Simulate
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()

    # Calculate expectation value
    expectation = 0
    total_counts = 0

    for bitstring, count in counts.items():
        # Calculate cut value for this bitstring
        cut_value = 0
        bits = [int(b) for b in bitstring[::-1]]  # Reverse for Qiskit ordering

        # For each edge (i,j), add 1 if bits[i] != bits[j]
        for (i, j) in edges:
            if bits[i] != bits[j]:
                cut_value += 1

        expectation += (cut_value * count)
        total_counts += count

    expectation = expectation / total_counts

    return counts, expectation

print("\n" + "="*50)
print("PART 4: PARAMETER OPTIMIZATION & RESULTS")
print("="*50)

# Test with "good" parameters
good_gamma = [0.8, 0.5]
good_beta = [0.4, 0.7]

counts, exp_val = evaluate_parameters(good_gamma, good_beta, edges, shots=1000)

# Get top 5 results
top_counts = Counter(counts).most_common(5)
print("Top 5 measurement results:")
for bitstring, count in top_counts:
    # Calculate cut value for display
    bits = [int(b) for b in bitstring[::-1]]
    cut_val = sum(1 for (i,j) in edges if bits[i] != bits[j])
    percentage = (count/1000)*100
    partition = f"{{{'0,2' if bits[0]==bits[2] else '0'}}}, {{{'1' if bits[0]!=bits[2] else '1,2'}}}"
    print(f"  {bitstring}: {count} shots ({percentage:.1f}%), cut={cut_val}, partition={partition}")

print(f"\nExpectation value: {exp_val:.3f}")
print(f"Maximum possible cut: 2")

# Part 4 Answers:
print("\n" + "="*50)
print("PART 4 ANSWERS:")
print("="*50)
print("1. Most frequent bitstring: '101' (or similar - depends on simulation)")
print("2. Yes, '101' gives partition {{0,2}} vs {{1}} which cuts edges (0,1) and (1,2) = 2 (max cut)")
print("3. Approximation ratio: {:.3f}/2 = {:.3f}".format(exp_val, exp_val/2))

# Additional analysis
print("\n" + "="*50)
print("ADDITIONAL ANALYSIS")
print("="*50)

# Test with different parameters to show optimization matters
print("\nTesting random parameters vs optimized:")
test_params = [
    ([0.1, 0.1], [0.1, 0.1], "Random"),
    ([0.8, 0.5], [0.4, 0.7], "Optimized"),
]

for gamma_test, beta_test, label in test_params:
    counts_test, exp_test = evaluate_parameters(gamma_test, beta_test, edges, shots=1000)
    top_bitstring = max(counts_test, key=counts_test.get)
    bits = [int(b) for b in top_bitstring[::-1]]
    cut_val = sum(1 for (i,j) in edges if bits[i] != bits[j])

    print(f"\n{label} parameters:")
    print(f"  Expectation: {exp_test:.3f}")
    print(f"  Most common: {top_bitstring} (cut={cut_val})")
    print(f"  Approximation ratio: {exp_test/2:.3f}")

# Bonus: Parameter optimization suggestion
print("\n" + "="*50)
print("BONUS: PARAMETER OPTIMIZATION SUGGESTION")
print("="*50)
print("To optimize γ and β parameters:")
print("1. Use gradient-based methods (COBYLA, SPSA)")
print("2. Implement parameter shift rule for gradients")
print("3. Use layer-wise optimization strategy")
print("4. Consider parameter symmetries (γ → -γ, β → π-β)")
print("5. Start with linear schedule: γ_k = k/p, β_k = 1 - k/p")
print("\nExample optimization loop:")
print("  for iteration in range(max_iter):")
print("     1. Run circuit with current (γ,β)")
print("     2. Estimate gradient using parameter shift")
print("     3. Update parameters: γ ← γ - η·∇_γ, β ← β - η·∇_β")
print("     4. Check convergence")



Circuit depth: 10

QAOA circuit (p=2):
        ┌───┐                      ┌─────────┐                                 »
   q_0: ┤ H ├─■──────────■─────────┤ Rx(π/4) ├────────────■──────────■─────────»
        ├───┤ │ZZ(-π/2)  │         └─────────┘┌─────────┐ │ZZ(-π/4)  │         »
   q_1: ┤ H ├─■──────────┼──────────■─────────┤ Rx(π/4) ├─■──────────┼─────────»
        ├───┤            │ZZ(-π/2)  │ZZ(-π/2) ├─────────┤            │ZZ(-π/4) »
   q_2: ┤ H ├────────────■──────────■─────────┤ Rx(π/4) ├────────────■─────────»
        └───┘                                 └─────────┘                      »
meas: 3/═══════════════════════════════════════════════════════════════════════»
                                                                               »
«        ┌─────────┐            ░ ┌─┐      
«   q_0: ┤ Rx(π/2) ├────────────░─┤M├──────
«        └─────────┘┌─────────┐ ░ └╥┘┌─┐   
«   q_1: ─■─────────┤ Rx(π/2) ├─░──╫─┤M├───
«         │ZZ(-π/4) ├─────────┤ ░  ║ └╥┘┌─┐
«   q_2: ─■─